In [12]:
#from keys import ml_prod_servicekey as ml_key
from keys import ml_servicekey_hole as ml_key
from ml_interface import MLFoundationClient, DefectCode
from database_interface import DBClient
from image_interface import ImageDBClient

In [7]:
# init the clients
db_client = DBClient("dummy")
ml_client = MLFoundationClient(ml_key, offline = True)
im_client = ImageDBClient()

NameError: name 'ml_key' is not defined

In [13]:
img_path = "test-photo2.jpeg"
img = open(img_path, "rb")

In [14]:
# use this key to upload to the database
key = im_client.upload(img_path)

'cb144563-73f2-417f-a600-eced70ef7438'

In [15]:
key = 'cb144563-73f2-417f-a600-eced70ef7438'
# for every model
client = MLFoundationClient(ml_servicekey_hole)
defects = client.modelPredictOne("Hole.jpg", "hole","HoleModel", DefectCode.Hole)

#unite the lists


#make the json
imageInfo= {
    'image1' : key,
    'image2' : key,
    'defects' : defects     
}
print(imageInfo)

{'image1': 'cb144563-73f2-417f-a600-eced70ef7438', 'image2': 'cb144563-73f2-417f-a600-eced70ef7438', 'defects': [<DefectCode.Hole: 5>]}


In [ ]:
db_client.saveMachineLearningResult(imageInfo, productExists=False)